In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from use_case.baseline import * 
from tests.eval import *

np.random.seed(1337)

N = 20
payoff_i = np.random.uniform(-10, 10, (N, N))
payoff_j = np.random.uniform(-10, 10, (N, N))

# Initialize environment
N_ACTIONS = payoff_i.shape[0]
N_AGENTS = 2000
env = BaselineEnvironment(N_AGENTS, payoff_i, payoff_j, total_games = 1)

# Actual Run

In [3]:
from models.model import *
from models.trainer import *
from torch.utils.tensorboard import SummaryWriter


In [4]:
# Configure the network here
parameters = ParameterSettings(
    n_agents = N_AGENTS,
    d_action = N_ACTIONS, 
    d_obs = env.obs_size, 
    d_traits = 1,
    d_beliefs = 1
)
parameters.device = "cuda" if torch.cuda.is_available() else "cpu"

model = Model(parameters)

In [5]:
equilibriua = find_pure_equilibria(payoff_i, payoff_j)

for eq in equilibriua:
    x, y = eq 
    a = (y[0] + y[1]) / 2

    print(x, a)

(18, 10) 9.42934343544164


In [6]:
evaluate_policy(model, env, 10)

np.float64(-0.004720429133506836)

In [ ]:
# Setup the training loop
training_parameters = TrainingParameters(
    outer_loops = 200,
    hypernet_training_loops= 10, 
    actor_training_loops = 30,

    actor_learning_rate= 2.5e-4,
    critic_learning_rate = 1e-3,
    hypernet_learning_rate = 2.5e-4,

    hypernet_jsd_threshold = 0.25,

    sampled_agents = N_AGENTS // 4,
    experience_buffer_size = 10,

)


train_model(model, env, training_parameters)
        

Epoch 0


Actor Training: 100%|██████████| 30/30 [00:08<00:00,  3.34it/s]


Epoch 1


Actor Training: 100%|██████████| 30/30 [00:09<00:00,  3.26it/s]


Epoch 2


...

Actor Training: 100%|██████████| 30/30 [00:08<00:00,  3.36it/s]


Epoch 44


Actor Training: 100%|██████████| 30/30 [00:08<00:00,  3.43it/s]


Epoch 45


Actor Training:  23%|██▎       | 7/30 [00:02<00:06,  3.43it/s]

# Heterogeneous Baseline

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from use_case.baseline import * 
from tests.eval import *
from models.model import *
from models.trainer import *


In [3]:
np.random.seed(1337)

N_AGENTS = 2000 
N_TYPES = 10
N_ACTIONS = 10
type_payoffs = np.random.uniform(-10, 10, (N_TYPES, N_TYPES, 2, N_ACTIONS, N_ACTIONS))
env = BaselineHeterogeneous(N_AGENTS, N_TYPES, type_payoffs)

In [4]:
# Configure the network here
parameters = ParameterSettings(
    n_agents = N_AGENTS,
    d_action = N_ACTIONS, 
    d_obs = env.obs_size, 
    d_traits = N_TYPES,
    d_beliefs = 1
    
)
parameters.device = "cuda" if torch.cuda.is_available() else "cpu"

model = Model(parameters)

In [5]:
# Setup the training loop
training_parameters = TrainingParameters(
    outer_loops = 200,
    hypernet_training_loops= 10, 
    actor_training_loops = 30,

    actor_learning_rate= 2.5e-4,
    critic_learning_rate = 1e-3,
    hypernet_learning_rate = 2.5e-4,

    hypernet_jsd_threshold = 0.25,

    sampled_agents = N_AGENTS // 4,
    experience_buffer_size = 10,

)


In [ ]:

train_model(model, env, training_parameters)

Epoch 0


Actor Training: 100%|██████████| 30/30 [02:04<00:00,  4.16s/it]


Epoch 1


Actor Training: 100%|██████████| 30/30 [02:03<00:00,  4.10s/it]


Epoch 2


...

Actor Training: 100%|██████████| 30/30 [02:29<00:00,  4.98s/it]


Epoch 28


Actor Training: 100%|██████████| 30/30 [02:21<00:00,  4.73s/it]


Epoch 29


Hypernet Loop:  50%|█████     | 5/10 [00:20<00:20,  4.18s/it]